In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image, ImageDraw

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Reading the video from drive and dividing it into frames with numbering as frame0,frame1,frame2....
 
# Read the video from specified path 
cam = cv2.VideoCapture("/content/gdrive/My Drive/tiny_yolov3/test_videos/new.mp4") 

currentframe = 0
  
while(True): 
      
    # reading from frame 
    ret,frame = cam.read() 
  
    if ret: 
        # if video is still left continue creating images 
        name = '/content/gdrive/My Drive/tiny_yolov3/test_videos/frames/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name) 
         # writing the extracted images 
        cv2.imwrite(name, frame) 
  
        # increasing counter so that it will 
        # show how many frames are created 
        currentframe += 1
    else: 
        break
  
# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows()

In [ ]:
# Code for creating text file containing paths to all the frames extracted from the video(will be used in detections)
file1 = open("images.txt","w")
for i in range (0,currentframe):
  a="/mydrive/tiny_yolov3/test_videos/frames/frame"+str(i)+".jpg"+"\n"
  file1.write(a)
file1.close

In [ ]:
# Code for detections with yolo on all the frames and creating json format for all the detections done frame by frame
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# (For tiny yolov3 do not do it only applicable for yolov4)
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!make

In [ ]:
# getting weights of yolo v4(tiny yolo v3)
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
pwd

'/content/darknet'

In [ ]:
# Code for detections on multiple images with there paths specified in images.txt file(A json file will be created that will contain detections of each frame)
!./darknet detector test cfg/coco.data cfg/yolov3-tiny.cfg /mydrive/tiny_yolov3/yolov3-tiny.weights /mydrive/tiny_yolov3/test_videos/frames_town_centre/frame32.jpg -ext_output
 
# !./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/tiny_yolov3/test_videos/frames/frame0.jpg -ext_output

# !./darknet detector test cfg/coco.data cfg/yolov3-tiny.cfg /mydrive/tiny_yolov3/yolov3-tiny.weights -ext_output -dont_show -out result.json < /mydrive/tiny_yolov3/test_videos/images_short.txt

# !./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -dont_show -out result.json < /mydrive/tiny_yolov3/test_videos/images.txt


In [ ]:
# Preprocessing made on json format and extracting the bounding box coordinates
f=open("/content/darknet/result.json")
data=json.load(f)

In [ ]:
# Function for finding eucledian distance between all the points in a given array
from scipy.spatial import distance
def compute_distance(midpoints,num):
  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1,num):
      if i!=j:
        dst = distance.euclidean(midpoints[i], midpoints[j])
        dist[i][j]=dst
  return dist

In [ ]:
# Function for returning an array based on the 2d array formed from compute_distance function
def find_closest(dist,num,thresh):
  p1=[]
  p2=[]
  d=[]
  for i in range(num):
    for j in range(i,num):
      if( (i!=j) & (dist[i][j]<=thresh)):
        p1.append(i)
        p2.append(j)
        d.append(dist[i][j])
  return p1,p2,d

In [ ]:
# Finding the matrix for conversion into perspective coordinates(only valid for oxford town center dataset)
src = np.float32([[1150,30], [1700,80], [1150,650], [100,475]])
dst = np.float32([[0,0], [1050,0], [1050, 570], [0, 570]])
matrix = cv2.getPerspectiveTransform(src, dst)

In [ ]:
# Function for transforming the midpoints into the birds eye coordinates
def transform_pts(point,prespective_transform):
  pts = np.float32([[point[0],point[1]]])
  pts=np.array([pts])
  bd_pnt = cv2.perspectiveTransform(pts, prespective_transform)[0][0]
  return ((int(bd_pnt[0]),int(bd_pnt[1]))) 

In [ ]:
# Function for making bounding boxes on all frames only around person class(green as well as red) 
z=len(data)
for i in range(0,z):
  m=data[i]["objects"]
  f=[]

  for j in range(0,len(m)):
    n=m[j]["relative_coordinates"]
    f.append(n)
 
  class_iden=[]
  for w in range(0,len(m)):
    l=m[w]["class_id"]
    class_iden.append(l)

  x_center=[]
  for k in range(0,len(data[i]["objects"])):
    b=f[k]["center_x"]
    x_center.append(b)

  y_center=[]
  for k in range(0,len(data[i]["objects"])):
    b=f[k]["center_y"]
    y_center.append(b)

  height=[]
  for k in range(0,len(data[i]["objects"])):
    b=f[k]["height"]
    height.append(b)

  width=[]
  for k in range(0,len(data[i]["objects"])):
    b=f[k]["width"]
    width.append(b)

  dict = {'x_center': x_center, 'y_center': y_center, 'height': height, 'width': width, 'class_id': class_iden}  
  df = pd.DataFrame(dict)
  # Multiplication with height and width of image(normalisation)
  df["x_top"]=(df["x_center"]-(df["width"]/2))*(1280)
  df["y_top"]=(df["y_center"]-(df["height"]/2))*(720)
  df["x_bottom"]=(df["x_center"]+(df["width"]/2))*(1280)
  df["y_bottom"]=(df["y_center"]+(df["height"]/2))*(720)

  df["x_bottom_mid"]=(df["x_top"]+df["x_bottom"])/2
  df["y_bottom_mid"]=df["y_bottom"]

  df=df[df["class_id"]==0]
  df=df.reset_index()
  df=df.drop(["x_center","y_center","height","width","class_id","index"],axis=1)
  
  num=len(df)

  # Computing midpoints of all the detection boxes
  midpoints=[]
  for k in range (0,num):
    mid_point=(int(df.loc[k]["x_bottom_mid"]),int(df.loc[k]["y_bottom_mid"]))
    midpoints.append(mid_point)

  # # converting the midpoints array into perspective coordinates
  # for i in range(0,num):
  #   midpoints[i]=transform_pts(midpoints[i],matrix)

  # computing the 2d 
  dist= compute_distance(midpoints,num)


  thresh=100
  p1,p2,d=find_closest(dist,num,thresh)
  w = pd.DataFrame({"p1":p1,"p2":p2,"dist":d})

  


  image=cv2.imread("/mydrive/tiny_yolov3/test_videos/frames/frame"+str(i)+".jpg")
  color=(0, 255, 0) 
  thickness=2
  for k in range(0,num):
    start_point=(int(df.loc[k]["x_top"]),int(df.loc[k]["y_top"]))
    end_point=(int(df.loc[k]["x_bottom"]),int(df.loc[k]["y_bottom"]))
    image=cv2.rectangle(image,start_point,end_point,color,thickness)

  risky = np.unique(p1+p2)
  for k in risky:
    x1,y1,x2,y2,z1,z2 = df.loc[k]
    image= cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
  
  u="/mydrive/tiny_yolov3/images_with_detection_boxes_new/frame"+str(i)+".jpg"
  cv2.imwrite(u,image)

In [ ]:
# Function for combining all the frames with bounding boxes in a single video
import os 
import re
frames = os.listdir('/mydrive/tiny_yolov3/images_with_detection_boxes_new')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
import cv2
frame_array=[]


for i in range(len(frames)):
    
    #reading each files
    img = cv2.imread('/content/gdrive/My Drive/tiny_yolov3/images_with_detection_boxes_new/'+frames[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)



In [ ]:
out = cv2.VideoWriter('sample_output_new.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 


In [ ]:
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()